# Implementing the Neural Network for Transition-Based Dependency Parsing

This notebook implements the feedforward neural network model described in the CS224n Lecture Notes, Part IV (Dependency Parsing 2, Winter 2019), specifically Section 1.4 and Figure 3.

**Goal:** To build a neural network that, given features extracted from a parser's state (stack, buffer, existing arcs), predicts the next optimal transition (e.g., SHIFT, LEFT-ARC, RIGHT-ARC) to take in order to construct a dependency tree.

**Focus:** This implementation focuses *only* on the neural network architecture itself. It does not implement the actual transition system logic (managing the stack, buffer, and arcs). We will simulate the input features that such a system would provide.

## 1. Setup and Imports

We'll use PyTorch to build the neural network.

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

## 2. Understanding the Input Features

The lecture notes describe extracting features from the current parser configuration `c = (σ, β, A)` (stack, buffer, arcs).

**Feature Types:**
1.  `Sword`: Specific words from the top of the stack, front of the buffer, and possibly children/dependents of words on the stack.
2.  `Stag`: Part-of-Speech (POS) tags corresponding to the words selected for `Sword`.
3.  `Slabel`: Dependency relation labels for arcs connected to the selected words (e.g., labels of children).

**Example Feature Set (from Page 4):**
*   `Sword`: 18 specific word positions (e.g., top 3 stack, top 3 buffer, children of top 2 stack, grandchildren).
*   `Stag`: 18 corresponding POS tags.
*   `Slabel`: 12 corresponding arc labels (for the children/grandchildren positions).

**Representation:**
*   Each actual word, tag, or label is mapped to a unique integer ID.
*   A special `NULL` token/ID is used for positions where no word/tag/label exists (e.g., empty stack/buffer, no children).
*   The input to the network for a given state `c` is a set of these integer IDs, corresponding to the pre-defined feature template.

## 3. Defining Model Parameters and Hyperparameters

We need to define the sizes of our vocabularies, the embedding dimensions, hidden layer size, and the number of possible output transitions.

In [ ]:
# Vocabulary sizes (including NULL token)
WORD_VOCAB_SIZE = 10000  # Example size
TAG_SET_SIZE = 50       # Example size (e.g., Penn Treebank tags + NULL)
LABEL_SET_SIZE = 40     # Example size (e.g., Universal Dependencies labels + NULL)

# Feature counts (based on the example in the notes)
NUM_WORD_FEATURES = 18
NUM_TAG_FEATURES = 18
NUM_LABEL_FEATURES = 12

# Dimensions
EMBEDDING_DIM = 50      # Dimension 'd' for all embeddings
HIDDEN_DIM = 200        # Size of the hidden layer

# Output size: Number of possible transitions
# Example: 1 SHIFT + (LABEL_SET_SIZE-1) LEFT-ARCs + (LABEL_SET_SIZE-1) RIGHT-ARCs 
# (Subtract 1 because NULL label isn't used for actual arcs)
NUM_TRANSITIONS = 1 + (LABEL_SET_SIZE - 1) + (LABEL_SET_SIZE - 1) 

# Training Hyperparameters (example)
LEARNING_RATE = 0.001
BATCH_SIZE = 64

## 4. The Neural Network Model Architecture

The model follows Figure 3:
1.  **Input Layer:** Takes integer IDs for the selected word, tag, and label features.
2.  **Embedding Layer:** Maps each integer ID to a dense vector using embedding matrices (`Ew`, `Et`, `El`).
3.  **Concatenation:** Concatenates all the retrieved embedding vectors into a single large input vector `[xw, xt, xl]`.
4.  **Hidden Layer:** Applies an affine transformation (`W1`, `b1`) followed by a non-linear activation function. The notes use `f(x) = x^3`.
   *   $h = (W_1^w x^w + W_1^t x^t + W_1^l x^l + b_1)^3$
   *   Alternatively, concatenate first: $h = (W_1 [x^w, x^t, x^l] + b_1)^3$
   We will implement the second version (concatenate first) for simplicity, which is equivalent if `W1` is structured appropriately.
5.  **Output Layer:** Applies another affine transformation (`W2`, `b2`) to produce scores (logits) for each possible transition.
   *   $logits = W_2 h + b_2$
6.  **Softmax Layer (Implicit in Loss):** A softmax function is applied to the logits to get probabilities for each transition. This is typically combined with the Cross-Entropy loss function during training.

In [ ]:
class NeuralDependencyParserModel(nn.Module):
    def __init__(self, word_vocab_size, tag_set_size, label_set_size, 
                 num_word_features, num_tag_features, num_label_features,
                 embedding_dim, hidden_dim, num_transitions):
        super(NeuralDependencyParserModel, self).__init__()

        # Embedding layers
        self.word_embedding = nn.Embedding(word_vocab_size, embedding_dim)
        self.tag_embedding = nn.Embedding(tag_set_size, embedding_dim)
        self.label_embedding = nn.Embedding(label_set_size, embedding_dim)

        # Calculate the total size of the concatenated embedding vector
        self.total_input_dim = (num_word_features + num_tag_features + num_label_features) * embedding_dim

        # Linear layers
        # Layer 1: Maps concatenated embeddings to hidden layer
        self.layer1 = nn.Linear(self.total_input_dim, hidden_dim)
        # Layer 2: Maps hidden layer to output logits (scores for each transition)
        self.layer2 = nn.Linear(hidden_dim, num_transitions)

        # Optional: Initialize weights (e.g., Xavier initialization)
        nn.init.xavier_uniform_(self.layer1.weight)
        nn.init.constant_(self.layer1.bias, 0.1)
        nn.init.xavier_uniform_(self.layer2.weight)
        nn.init.constant_(self.layer2.bias, 0.1)

    def forward(self, word_features, tag_features, label_features):
        """
        Performs the forward pass of the network.
        
        Args:
            word_features (torch.Tensor): Tensor of word indices (batch_size, num_word_features).
            tag_features (torch.Tensor): Tensor of tag indices (batch_size, num_tag_features).
            label_features (torch.Tensor): Tensor of label indices (batch_size, num_label_features).
            
        Returns:
            torch.Tensor: Logits for each possible transition (batch_size, num_transitions).
        """
        # 1. Get embeddings for all features
        # Shape: (batch_size, num_features, embedding_dim)
        word_embeds = self.word_embedding(word_features)
        tag_embeds = self.tag_embedding(tag_features)
        label_embeds = self.label_embedding(label_features)

        # 2. Flatten embeddings before concatenation
        # Shape: (batch_size, num_features * embedding_dim)
        batch_size = word_features.size(0)
        word_flat = word_embeds.view(batch_size, -1)
        tag_flat = tag_embeds.view(batch_size, -1)
        label_flat = label_embeds.view(batch_size, -1)

        # 3. Concatenate all feature embeddings
        # Shape: (batch_size, total_input_dim)
        combined_embeds = torch.cat((word_flat, tag_flat, label_flat), dim=1)

        # 4. Hidden Layer: Affine transformation + Cubic Activation
        # Shape: (batch_size, hidden_dim)
        h = self.layer1(combined_embeds)
        h_activated = torch.pow(h, 3) # Cubic activation as per Figure 3
        # Alternatively, use other activations like ReLU: 
        # h_activated = torch.relu(h)

        # 5. Output Layer: Affine transformation to get logits
        # Shape: (batch_size, num_transitions)
        logits = self.layer2(h_activated)

        return logits

## 5. Simulating Input Data and Training Step

Since we don't have a real parser state generator, we will create random dummy input data (feature indices) and target transitions to demonstrate how the model would be trained.

In [ ]:
# Instantiate the model
model = NeuralDependencyParserModel(
    word_vocab_size=WORD_VOCAB_SIZE,
    tag_set_size=TAG_SET_SIZE,
    label_set_size=LABEL_SET_SIZE,
    num_word_features=NUM_WORD_FEATURES,
    num_tag_features=NUM_TAG_FEATURES,
    num_label_features=NUM_LABEL_FEATURES,
    embedding_dim=EMBEDDING_DIM,
    hidden_dim=HIDDEN_DIM,
    num_transitions=NUM_TRANSITIONS
)

# Define loss function and optimizer
# CrossEntropyLoss expects raw logits from the model and target class indices.
# It internally applies log-softmax and NLLLoss.
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

# --- Simulate a batch of data ---
# In a real scenario, these would be extracted from parser states.
# We use random integers within the valid range for IDs.

# Input feature indices (Batch Size, Num Features)
dummy_word_indices = torch.randint(0, WORD_VOCAB_SIZE, (BATCH_SIZE, NUM_WORD_FEATURES), dtype=torch.long)
dummy_tag_indices = torch.randint(0, TAG_SET_SIZE, (BATCH_SIZE, NUM_TAG_FEATURES), dtype=torch.long)
dummy_label_indices = torch.randint(0, LABEL_SET_SIZE, (BATCH_SIZE, NUM_LABEL_FEATURES), dtype=torch.long)

# Target transitions (Batch Size) - indices from 0 to NUM_TRANSITIONS-1
dummy_target_transitions = torch.randint(0, NUM_TRANSITIONS, (BATCH_SIZE,), dtype=torch.long)

# --- Perform a single training step ---

# 1. Zero gradients
optimizer.zero_grad()

# 2. Forward pass: Get logits from the model
logits = model(dummy_word_indices, dummy_tag_indices, dummy_label_indices)

# 3. Calculate loss
loss = criterion(logits, dummy_target_transitions)

# 4. Backward pass: Calculate gradients
loss.backward()

# 5. Update weights
optimizer.step()

# Print the loss for this step
print(f'Simulated training step finished.')
print(f'Loss: {loss.item():.4f}')

# Optionally, look at the output shape and some logits
print(f'\nLogits shape: {logits.shape}') # Should be (BATCH_SIZE, NUM_TRANSITIONS)
# print(f'Example logits: {logits[0]}')

# To get probabilities (e.g., for prediction during parsing):
probabilities = torch.softmax(logits, dim=1)
print(f'\nProbabilities shape: {probabilities.shape}')
# print(f'Example probabilities: {probabilities[0]}')
# The predicted transition would be the one with the highest probability:
predicted_transitions = torch.argmax(probabilities, dim=1)
print(f'Example predicted transition index for first item in batch: {predicted_transitions[0].item()}')

## 6. Summary and Next Steps

This notebook successfully implements the feedforward neural network component of a transition-based dependency parser as described in the CS224n notes.

**Key Takeaways:**
*   Feature templates define which words, tags, and labels from the parser state are used as input.
*   Embedding layers convert sparse ID representations into dense vectors.
*   Concatenated embeddings form the input to a standard feedforward network.
*   The network uses non-linear activations (like the specified cubic function or common ones like ReLU/Tanh) in the hidden layer.
*   The output layer produces scores (logits) for each possible parser transition.
*   Cross-Entropy loss is suitable for training this multi-class classification problem.

**To build a complete parser, one would need to:**
1.  Implement the actual arc-standard (or other) transition system (managing stack $\sigma$, buffer $\beta$, and arc set $A$).
2.  Define a precise feature extraction function that maps a parser state `c = (σ, β, A)` to the required `word_features`, `tag_features`, and `label_features` tensors.
3.  Create or obtain a training dataset of sentences annotated with gold dependency trees.
4.  Generate gold transition sequences from the gold trees (an "oracle").
5.  Train the neural network model using the extracted features and corresponding gold transitions from the oracle.
6.  Implement the parsing algorithm: Start from an initial state, repeatedly use the trained model to predict the highest-scoring valid transition, apply the transition to update the state, and continue until a terminal state is reached.